In [ ]:
import langchain, backoff, openai, requests, urllib.parse
import tkinter as tk
from tkinter import filedialog
import docx2txt
import PyPDF2
import pandas as pd
import pdfplumber

from langchain import SerpAPIWrapper
from langchain.utilities.wolfram_alpha import WolframAlphaAPIWrapper
from langchain.tools import HumanInputRun
from apikeys import WolframAplha_KEY, SerpAPI_KEY
from langchain.agents import Tool
from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.chat_models import ChatOpenAI
from apikeys import OPENAI_KEY
import os

os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")
llm = OpenAI(temperature=0)
os.environ["WOLFRAM_ALPHA_APPID"] = WolframAplha_KEY
os.environ["SERPAPI_API_KEY"] = ""

search = SerpAPIWrapper()
wolfram = WolframAlphaAPIWrapper()

In [ ]:
import re
def read_file():
    # Create a root Tk window and hide it
    root = tk.Tk()
    root.withdraw()

    # Open a file dialog and get the path to the selected file
    file_path = filedialog.askopenfilename()

    # Get the file extension
    _, file_extension = os.path.splitext(file_path)

    if file_extension == '.txt':
        # Read .txt file
        with open(file_path, 'r') as file:
            text = file.read()
 
    elif file_extension == '.docx':
        # Read .docx file
        text = docx2txt.process(file_path)

    elif file_extension == '.pdf':
        # Read .pdf file
        with pdfplumber.open(file_path) as pdf:
            first_page = pdf.pages[0]
            text = first_page.extract_text()

    elif file_extension in ['.xls', '.xlsx']:
        # Read Excel file
        df = pd.read_excel(file_path)
        text = df.to_string()

    else:
        print(f"Unsupported file extension: {file_extension}")
        text = ""

    return text

def calculate_score(true_count, total_count):
    """Calculate the verification score."""
    if total_count == 0:
        return 0
    return true_count / total_count


In [ ]:
import re
import openai

def breakdown_answer(answer):
    # Split the input paragraph into sentences
    sentences = re.split(r'\.', answer)

    # Remove any empty strings resulting from split
    sentences = [s.strip() for s in sentences if s.strip()]

    chunks = []
    # Now iterate over each sentence and send to OpenAI API
    for sentence in sentences:
        prompt = f"""You are a highly advanced language model programmed to decompose complex text into simpler factoids. Your primary task is to dissect larger narratives into standalone pieces of information, with each piece containing a single fact that can be independently verified. 

    Your goal is to retain the original meaning and context of the information without making any corrections or alterations. This includes both explicit details and implied facts present in the original text. Additionally, you should be able to identify and illustrate dependencies between factoids, creating a hierarchy that reflects the nested structure of the facts.

    Even if the information appears incorrect, your task is not to correct it, but to reproduce it faithfully in a simpler, structured form that can be easily verified or refuted.

    Apply this to the following statement:
    {sentence}
    Output only the sentences - one in each line.
        """
        response = openai.Completion.create(
            engine="text-davinci-003",
            prompt=prompt,
            temperature=0,
            max_tokens=200
        )
        raw_chunks = response.choices[0].text.strip().split("\n")

        # Split raw chunks by full stop and comma
        for chunk in raw_chunks:
            sub_chunks = re.split(r'[.,;]', chunk)
            # Filter out chunks that are purely numerical and add to the list
            chunks.extend([sub.strip() for sub in sub_chunks if sub.strip() and not sub.strip().isdigit()])

    return chunks


In [ ]:
def calculate_score(true_count, total_count):
    """Calculate the verification score."""
    if total_count == 0:
        return 0
    return true_count / total_count


In [ ]:
from tqdm import tqdm
import openai
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from llama_index import SummaryIndex, RssReader
from langchain.vectorstores import Qdrant
from langchain.document_loaders import TextLoader
from llama_index import SummaryIndex, SimpleWebPageReader  

def calculate_score(true_count, total_count):
    if total_count == 0:
        return 0
    return true_count / total_count

def get_relevant_passages_qdrant(qdrant, fact_chunk):
    found_docs = qdrant.max_marginal_relevance_search(fact_chunk, k=3, fetch_k=10)
    return [doc.page_content for doc in found_docs]

def check_fact_in_web_contexts(question, answer, answer_chunks, web_url):
    # Load web document
    documents = SimpleWebPageReader(html_to_text=True).load_data([web_url])
    index = SummaryIndex.from_documents(documents)
    query_engine = index.as_query_engine()
    
    true_count = 0
    fact_statuses = []
    
    for i, chunk in enumerate(tqdm(answer_chunks, desc="Verifying facts", unit="fact")):
        # Get relevant passages from web document
        response = query_engine.query(chunk)
        
        if not response:
            fact_statuses.append("Unverifiable")
            continue
        
        print(f"Fact {i+1} to be checked: {chunk}")
        print(f"Relevant Passage: {response}")

        prompt = f"""Factual Information: {response}
        Question: {question}
        Is the following statement correct? {chunk}
        Answer "True" or "False" with a brief explanation."""
        
        response = openai.Completion.create(
            engine="text-davinci-003",
            prompt=prompt,
            temperature=0,
            max_tokens=100
        )
        
        response_text = response.choices[0].text.strip()
        print(f"Verification Result: {response_text}")
        
        if "True" in response_text:
            true_count += 1

        score = calculate_score(true_count, 1)
        fact_statuses.append("True" if score > 0.5 else "False")
        true_count = 0
    
    print("Final Fact Verification Results:")
    for i, status in enumerate(fact_statuses):
        print(f"Fact {i+1}: {answer_chunks[i]} - {status}")

def main():
    question = input("Please enter your question: ")
    answer = input("Please enter your answer: ")
    web_url = input("Please enter the web URL for context: ")
    
   
    answer_chunks = breakdown_answer(answer)
    
    print(f"\nTotal number of facts identified: {len(answer_chunks)}\n")
    
    check_fact_in_web_contexts(question, answer, answer_chunks, web_url)

if __name__ == "__main__":
    main()
